# **MixSync: Song Key & BPM Finder**
With this application, you can accurately find and sort songs by key and BPM,
curated with meticulous verification, and filter them by various attributes for a seamless music-matching experience.

In [2]:
from google.colab import drive
drive.mount ('/gdrive')

Mounted at /gdrive


In [14]:
# @title Western: Searching Songs
import pandas as pd
import time
pd.options.mode.chained_assignment = None

# Delay function for animation purposes
def delay():
    time.sleep(0.5)


# Find songs within specified BPM range
def bpm_finder(database, bpm, bpm_thresh):
    delay()
    print(f"Finding songs that are ±{bpm_thresh} BPM around {bpm} BPM...")
    bpm_min = bpm - bpm_thresh
    bpm_max = bpm + bpm_thresh
    find_bpm = database[(database['BPM'].between(bpm_min, bpm_max))]
    return find_bpm


# Find songs within specified key range
def key_finder(find_bpm, key, key_thresh, key_list):
    delay()
    print(f"Finding songs that are {key_thresh} keys apart from {key}...")
    key_index = key_list.index(key)
    surrounding_keys = []
    for counter in range(-key_thresh, key_thresh + 1):
        # if 0 <= key_index + counter < len(key_list):
        surrounding_keys.append(key_list[key_index + counter])
    find_keybpm = find_bpm[find_bpm['KEY'].isin(surrounding_keys)]
    return find_keybpm


# Function to generate sorted BPM values
def bpm_range(bpm, bpm_thresh):
    # Calculate the range of numbers around the midpoint
    start = bpm - bpm_thresh
    end = bpm + bpm_thresh

    # Generate the array of numbers within the range
    numbers = list(range(start, end + 1))

    # Sort the numbers based on their proximity to the midpoint
    numbers.sort(key=lambda x: abs(x - bpm))
    return numbers


# Function to create a custom sort order
def sort_bpm(df, midpoint, gap):
    sorted_bpm = bpm_range(midpoint, gap)
    sort_order = {bpm: index for index, bpm in enumerate(sorted_bpm)}
    df['sort_order'] = df['BPM'].map(sort_order)
    df = df.sort_values(by='sort_order').drop(columns=['sort_order'])
    return df


# Function to generate sorted keys based on proximity to the midpoint key
def key_range(key, key_thresh, key_list):
    midpoint_index = key_list.index(key)
    # Include the midpoint key first
    sorted_keys = [key]

    # Add keys within the gap range, alternating between lower and upper proximity
    for i in range(0, key_thresh + 1):
        lower_index = (midpoint_index - i) % len(key_list)
        upper_index = (midpoint_index + i) % len(key_list)

        sorted_keys.append(key_list[lower_index])
        sorted_keys.append(key_list[upper_index])

    # Filter out any duplicates while preserving order
    seen = set()
    sorted_keys = [mkey for mkey in sorted_keys if not (mkey in seen or seen.add(mkey))]

    return sorted_keys


# Function to create a custom sort order for the DataFrame
def sort_keys(df, key, key_thresh, key_list):
    sorted_keys = key_range(key, key_thresh, key_list)
    sort_order = {mkey: index for index, mkey in enumerate(sorted_keys)}
    df.loc[:, 'sort_order'] = df['KEY'].map(sort_order)
    df = df.sort_values(by='sort_order').drop(columns=['sort_order'])
    return df


# Sort database by specified column
def sorting(database, db_name, sort_column, sort_order):
    if sort_order == "Ascending":
        sorted_df = database.sort_values(by=sort_column)
    elif sort_order == "Descending":
        sorted_df = database.sort_values(by=sort_column, ascending=False)
    sorted_df.to_csv(db_name, index=False)
    return sorted_df


def song_match(database, bpm, key, bpm_thresh, key_thresh, key_list, columns_to_display, div):
    # Adjust bpm and bpm_thresh if bpm is not in the range
    if bpm not in range(70, 140):
        if bpm >= 140:
            bpm /= 2
            bpm_thresh /= 2
        elif bpm < 70:
            bpm *= 2
            bpm_thresh *= 2

    sort_db = sort_bpm(database, bpm, bpm_thresh)

    # Function calls: Find songs
    find_bpm = bpm_finder(sort_db, bpm, bpm_thresh)
    find_keybpm = key_finder(find_bpm, key, key_thresh, key_list)
    sorted_keybpm = sort_keys(find_keybpm, key, key_thresh, key_list)

    # Display results
    delay()
    print(f"{div} SONGS {div}")
    print(sorted_keybpm[columns_to_display])
    print(f"{div}======={div}")


# MAIN PROGRAM
if __name__ == "__main__":
  div = "=" * 15
  print(f"{div} WELCOME TO MIXSYNC {div}")
  print("""With this application, you can accurately find and sort songs by key and BPM,
  curated with meticulous verification, and filter them by various attributes for a
  seamless music-matching experience.""")

  surrounding_keys = []
  process_opt = [1, 2, 3, 4]

  # List of keys
  key_list = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]

  db_title = 'Western'
  db_name = 'western.csv'

  # @markdown Folder Location
  location = "/gdrive/My Drive/MixSync" # @param {type:"string"}
  location = location + "/western.csv"

  database = pd.read_csv(location)
  col_disp = ['ARTIST', 'TITLE', 'KEY', 'BPM']
  pd.set_option('max_colwidth', 30)
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)

  avail_column = database.columns.values

  while True:
      search_field = "TITLE" # @param ["ARTIST", "CONTRIBUTING ARTIST", "TITLE", "KEY", "BPM", "TYPE", "YEAR"]
      if search_field in avail_column:
          search_term = "Shake it" # @param {type:"string"}
          sort_order = "Ascending" # @param ["Ascending", "Descending"]
          sort_db = sorting(database, db_name, search_field, sort_order)
          break
      else:
          print("Invalid specified column.")
          continue

  # Perform the search
  searched = sort_db[sort_db[search_field].astype(str).str.contains(str(search_term.lower()), case=False, na=False)]

  # Display results
  delay()
  print(f"{div} SONGS {div}")
  print(searched[col_disp])
  print(f"{div}======={div}")

  if searched is not None and not searched.empty:
      for counter in range(len(searched.index)):
          searched_title = searched.iloc[counter]['TITLE']
          print(f"Searching songs that would match '{searched.iloc[counter]['TITLE']}' by '{searched.iloc[counter]['ARTIST']}'... ")
          bpm = int(searched.iloc[counter]['BPM'])
          key = searched.iloc[counter]['KEY']
          break

      # User input: BPM and Key Threshold
      while True:
          try:
              # @markdown BPM Threshold:
              # @markdown The allowable deviation from the specified BPM to include other songs that have a BPM close to the specified value.
              bpm_thresh = 5 # @param {type:"slider", min:0, max:35, step:1}
              break
          except ValueError:
              print("Input is not a valid number.")

      while True:
          try:
              # @markdown Key Threshold:
              # @markdown The number of musical keys away from the specified key that you are willing to consider when searching for songs.
              key_thresh = 0 # @param {type:"slider", min:0, max:6, step:1}
              break
          except ValueError:
              print("Input is not a valid number.")



      song_match(database, bpm, key, bpm_thresh, key_thresh, key_list, col_disp, div)

  else:
      print("No search performed.")

=============== WELCOME TO MIXSYNC ===============
With this application, you can accurately find and sort songs by key and BPM,
  curated with meticulous verification, and filter them by various attributes for a
  seamless music-matching experience.
=============== SONGS ===============
            ARTIST         TITLE KEY   BPM
2284  Taylor Swift  Shake It Off   G  80.0
Searching songs that would match 'Shake It Off' by 'Taylor Swift'... 
Finding songs that are ±5 BPM around 80 BPM...
Finding songs that are 0 keys apart from G...
=============== SONGS ===============
                   ARTIST                          TITLE KEY   BPM
6            Taylor Swift               ...Ready For It?   G  80.0
682       Imagine Dragons                        Digital   G  83.5
3053                 ZAYN                          tRuTh   G  85.0
430               Rihanna            California King Bed   G  85.0
1425         Shawn Mendes                    Kid In Love   G  85.0
3041        Billie Eil

In [6]:
# @title Western: Matching Songs
import pandas as pd
import time
pd.options.mode.chained_assignment = None

# Delay function for animation purposes
def delay():
    time.sleep(0.5)


# Find songs within specified BPM range
def bpm_finder(database, bpm, bpm_thresh):
    delay()
    print(f"Finding songs that are ±{bpm_thresh} BPM around {bpm} BPM...")
    bpm_min = bpm - bpm_thresh
    bpm_max = bpm + bpm_thresh
    find_bpm = database[(database['BPM'].between(bpm_min, bpm_max))]
    return find_bpm


# Find songs within specified key range
def key_finder(find_bpm, key, key_thresh, key_list):
    delay()
    print(f"Finding songs that are {key_thresh} keys apart from {key}...")
    key_index = key_list.index(key)
    surrounding_keys = []
    for counter in range(-key_thresh, key_thresh + 1):
        # if 0 <= key_index + counter < len(key_list):
        surrounding_keys.append(key_list[key_index + counter])
    find_keybpm = find_bpm[find_bpm['KEY'].isin(surrounding_keys)]
    return find_keybpm


# Function to generate sorted BPM values
def bpm_range(bpm, bpm_thresh):
    # Calculate the range of numbers around the midpoint
    start = bpm - bpm_thresh
    end = bpm + bpm_thresh

    # Generate the array of numbers within the range
    numbers = list(range(start, end + 1))

    # Sort the numbers based on their proximity to the midpoint
    numbers.sort(key=lambda x: abs(x - bpm))
    return numbers


# Function to create a custom sort order
def sort_bpm(df, midpoint, gap):
    sorted_bpm = bpm_range(midpoint, gap)
    sort_order = {bpm: index for index, bpm in enumerate(sorted_bpm)}
    df['sort_order'] = df['BPM'].map(sort_order)
    df = df.sort_values(by='sort_order').drop(columns=['sort_order'])
    return df


# Function to generate sorted keys based on proximity to the midpoint key
def key_range(key, key_thresh, key_list):
    midpoint_index = key_list.index(key)
    # Include the midpoint key first
    sorted_keys = [key]

    # Add keys within the gap range, alternating between lower and upper proximity
    for i in range(0, key_thresh + 1):
        lower_index = (midpoint_index - i) % len(key_list)
        upper_index = (midpoint_index + i) % len(key_list)

        sorted_keys.append(key_list[lower_index])
        sorted_keys.append(key_list[upper_index])

    # Filter out any duplicates while preserving order
    seen = set()
    sorted_keys = [mkey for mkey in sorted_keys if not (mkey in seen or seen.add(mkey))]

    return sorted_keys


# Function to create a custom sort order for the DataFrame
def sort_keys(df, key, key_thresh, key_list):
    sorted_keys = key_range(key, key_thresh, key_list)
    sort_order = {mkey: index for index, mkey in enumerate(sorted_keys)}
    df.loc[:, 'sort_order'] = df['KEY'].map(sort_order)
    df = df.sort_values(by='sort_order').drop(columns=['sort_order'])
    return df


# Sort database by specified column
def sorting(database, db_name, sort_column):
    sort_order = input("Ascending or Descending? [A/D]: ")
    if sort_order.lower() == "a":
        sorted_df = database.sort_values(by=sort_column)
    elif sort_order.lower() == "d":
        sorted_df = database.sort_values(by=sort_column, ascending=False)
    sorted_df.to_csv(db_name, index=False)
    return sorted_df


def search_song(database, db_name, columns_to_display, div):
    avail_column = database.columns.values
    print(f"{div} SEARCH FIELDS {div}")
    for i in range(0, len(avail_column), 2):
        if avail_column[i]:  # Check if the current column is True
            if i + 1 < len(avail_column) and avail_column[i + 1]:  # Check if the next column exists and is True
                print("{:<15} {:<15}".format(f"[{i}] {avail_column[i]}", f"[{i + 1}] {avail_column[i + 1]}"))
            elif i + 1 >= len(avail_column) or not avail_column[i + 1]:  # Check if the next column does not exist or is not True
                print(f"[{i}] {avail_column[i]}")

    while True:
        search_field = int(input("Search by? [#]: "))
        if search_field in range(len(avail_column)):
            search_field = avail_column[search_field]
            search_term = input("Search Term: ").lower()
            sort_db = sorting(database, db_name, search_field)
            break
        else:
            print("Invalid specified column.")
            continue

    # Perform the search
    search_results = sort_db[sort_db[search_field].astype(str).str.contains(str(search_term), case=False, na=False)]

    # Display results
    delay()
    print(f"{div} SONGS {div}")
    print(search_results[columns_to_display])
    print(f"{div}======={div}")

    return search_results


def add_song(database, db_name, db_title, key_list):
    print(f"You are adding a new record in the '{db_title}' database...")
    avail_column = database.columns.values
    print(f"SEARCH FIELDS: {avail_column}")

    while True:
        song_dl = {}
        for counter in range(len(avail_column)):
            while True:
                if avail_column[counter] != 'TYPE':
                    song_data = input(f"Enter {avail_column[counter]}: ")
                else:
                    song_data = db_title

                if avail_column[counter] == 'KEY':
                    if song_data not in key_list:
                        print("Invalid key.")
                        continue
                    else:
                        break

                elif avail_column[counter] == 'BPM':
                    song_data = int(song_data)
                    while True:
                        if song_data < 70:
                            song_data *= 2
                        elif song_data >= 140:
                            song_data /= 2
                        elif 70 < song_data < 140:
                            break
                    break

                elif avail_column[counter] == 'YEAR':
                    song_data = int(song_data)
                    while True:
                        if 0 < song_data:
                            break
                        else:
                            print("Invalid year.")
                            continue
                    break

                else:
                    break

            song_dl[avail_column[counter]] = song_data
            print(song_dl)

        for item in range(len(avail_column)):
            current_col = avail_column[item]
            current_data = song_dl[current_col]
            print(f"[{item + 1}] {current_col}: {current_data}")

        new_data = pd.DataFrame([song_dl])
        data_check = input("Is this correct [Y/N]: ").lower()

        if data_check == "y":
            updated_df = pd.concat([database, new_data], ignore_index=True)
            updated_df.to_csv(db_name, index=False)

        else:
            data_confirm = input("Discard the current record? [Y/N]:").lower()
            if data_confirm == "y":
                print("Data discarded.")
                break
            elif data_confirm == "n":
                updated_df = pd.concat([database, new_data], ignore_index=True)
                updated_df.to_csv(db_name, index=False)

        # User input: Ask if the user wants to add another song
        option = input("Do you still want to add another song [Y/N]: ").lower()
        if option != "y":
            delay()
            print("Thank you for using MixSync!")
            break


# MAIN PROGRAM
if __name__ == "__main__":
  div = "=" * 15
  print(f"{div} WELCOME TO MIXSYNC {div}")
  print("""With this application, you can accurately find and sort songs by key and BPM,
  curated with meticulous verification, and filter them by various attributes for a
  seamless music-matching experience.""")

  surrounding_keys = []
  process_opt = [1, 2, 3, 4]

  # List of keys
  key_list = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]

  db_title = 'Western'
  db_name = 'western.csv'

  # @markdown Folder Location
  location = "/gdrive/My Drive/MixSync" # @param {type:"string"}
  location = location + "/western.csv"

  database = pd.read_csv(location)
  col_disp = ['ARTIST', 'TITLE', 'KEY', 'BPM']
  pd.set_option('max_colwidth', 30)
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)


  # User input: BPM, Key, BPM and Key Threshold
  while True:
      try:
          # @markdown BPM: Beats per Minute
          bpm = 105 # @param {type:"slider", min:70, max:139, step:1}
          if bpm not in range(70, 140):
              raise ValueError("BPM must be between 70 and 139.")
          break
      except ValueError:
          print("Input is not a valid number or outside the range.")

  while True:
      # @markdown Key: The pitch around which the song revolves melodically
      key = "C" # @param ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
      if key in key_list:
          break
      else:
          print("Input is not a valid key.")

  while True:
      try:
          # @markdown BPM Threshold:
          # @markdown The allowable deviation from the specified BPM to include other songs that have a BPM close to the specified value.
          bpm_thresh = 5 # @param {type:"slider", min:0, max:35, step:1}
          break
      except ValueError:
          print("Input is not a valid number.")

  while True:
      try:
          # @markdown Key Threshold:
          # @markdown The number of musical keys away from the specified key that you are willing to consider when searching for songs.
          key_thresh = 0 # @param {type:"slider", min:0, max:6, step:1}
          break
      except ValueError:
          print("Input is not a valid number.")

  # Adjust bpm and bpm_thresh if bpm is not in the range
  if bpm not in range(70, 140):
      if bpm >= 140:
          bpm /= 2
          bpm_thresh /= 2
      elif bpm < 70:
          bpm *= 2
          bpm_thresh *= 2

  sort_db = sort_bpm(database, bpm, bpm_thresh)

  # Function calls: Find songs
  find_bpm = bpm_finder(sort_db, bpm, bpm_thresh)
  find_keybpm = key_finder(find_bpm, key, key_thresh, key_list)
  sorted_keybpm = sort_keys(find_keybpm, key, key_thresh, key_list)

  # Display results
  delay()
  print(f"{div} SONGS {div}")
  print(sorted_keybpm[col_disp])
  print(f"{div}======={div}")

=============== WELCOME TO MIXSYNC ===============
With this application, you can accurately find and sort songs by key and BPM,
  curated with meticulous verification, and filter them by various attributes for a
  seamless music-matching experience.
Finding songs that are ±5 BPM around 105 BPM...
Finding songs that are 0 keys apart from C...
=============== SONGS ===============
                        ARTIST                          TITLE KEY     BPM
2375            Natalie LaRose                       Somebody   C  105.00
588             Britney Spears                       Criminal   C  108.00
2979           Imagine Dragons                    Working Man   C  101.00
1893             Ariana Grande                  Off The Table   C  101.00
1931             Ariana Grande                         Only 1   C  101.00
2376                  The 1975                  Somebody Else   C  101.00
1842                 Dua Lipa                     No Goodbyes   C  109.00
1118               Nicki 

In [18]:
# @title Western: Adding Songs
import pandas as pd
import time
pd.options.mode.chained_assignment = None

# Delay function for animation purposes
def delay():
    time.sleep(0.5)


# Find songs within specified BPM range
def bpm_finder(database, bpm, bpm_thresh):
    delay()
    print(f"Finding songs that are ±{bpm_thresh} BPM around {bpm} BPM...")
    bpm_min = bpm - bpm_thresh
    bpm_max = bpm + bpm_thresh
    find_bpm = database[(database['BPM'].between(bpm_min, bpm_max))]
    return find_bpm


# Find songs within specified key range
def key_finder(find_bpm, key, key_thresh, key_list):
    delay()
    print(f"Finding songs that are {key_thresh} keys apart from {key}...")
    key_index = key_list.index(key)
    surrounding_keys = []
    for counter in range(-key_thresh, key_thresh + 1):
        # if 0 <= key_index + counter < len(key_list):
        surrounding_keys.append(key_list[key_index + counter])
    find_keybpm = find_bpm[find_bpm['KEY'].isin(surrounding_keys)]
    return find_keybpm


# Function to generate sorted BPM values
def bpm_range(bpm, bpm_thresh):
    # Calculate the range of numbers around the midpoint
    start = bpm - bpm_thresh
    end = bpm + bpm_thresh

    # Generate the array of numbers within the range
    numbers = list(range(start, end + 1))

    # Sort the numbers based on their proximity to the midpoint
    numbers.sort(key=lambda x: abs(x - bpm))
    return numbers


# Function to create a custom sort order
def sort_bpm(df, midpoint, gap):
    sorted_bpm = bpm_range(midpoint, gap)
    sort_order = {bpm: index for index, bpm in enumerate(sorted_bpm)}
    df['sort_order'] = df['BPM'].map(sort_order)
    df = df.sort_values(by='sort_order').drop(columns=['sort_order'])
    return df


# Function to generate sorted keys based on proximity to the midpoint key
def key_range(key, key_thresh, key_list):
    midpoint_index = key_list.index(key)
    # Include the midpoint key first
    sorted_keys = [key]

    # Add keys within the gap range, alternating between lower and upper proximity
    for i in range(0, key_thresh + 1):
        lower_index = (midpoint_index - i) % len(key_list)
        upper_index = (midpoint_index + i) % len(key_list)

        sorted_keys.append(key_list[lower_index])
        sorted_keys.append(key_list[upper_index])

    # Filter out any duplicates while preserving order
    seen = set()
    sorted_keys = [mkey for mkey in sorted_keys if not (mkey in seen or seen.add(mkey))]

    return sorted_keys


# Function to create a custom sort order for the DataFrame
def sort_keys(df, key, key_thresh, key_list):
    sorted_keys = key_range(key, key_thresh, key_list)
    sort_order = {mkey: index for index, mkey in enumerate(sorted_keys)}
    df.loc[:, 'sort_order'] = df['KEY'].map(sort_order)
    df = df.sort_values(by='sort_order').drop(columns=['sort_order'])
    return df


# Sort database by specified column
def sorting(database, db_name, sort_column, sort_order):
    if sort_order == "Ascending":
        sorted_df = database.sort_values(by=sort_column)
    elif sort_order == "Descending":
        sorted_df = database.sort_values(by=sort_column, ascending=False)
    sorted_df.to_csv(db_name, index=False)
    return sorted_df


def song_match(database, bpm, key, bpm_thresh, key_thresh, key_list, columns_to_display, div):
    # Adjust bpm and bpm_thresh if bpm is not in the range
    if bpm not in range(70, 140):
        if bpm >= 140:
            bpm /= 2
            bpm_thresh /= 2
        elif bpm < 70:
            bpm *= 2
            bpm_thresh *= 2

    sort_db = sort_bpm(database, bpm, bpm_thresh)

    # Function calls: Find songs
    find_bpm = bpm_finder(sort_db, bpm, bpm_thresh)
    find_keybpm = key_finder(find_bpm, key, key_thresh, key_list)
    sorted_keybpm = sort_keys(find_keybpm, key, key_thresh, key_list)

    # Display results
    delay()
    print(f"{div} SONGS {div}")
    print(sorted_keybpm[columns_to_display])
    print(f"{div}======={div}")


# MAIN PROGRAM
if __name__ == "__main__":
  div = "=" * 15
  print(f"{div} WELCOME TO MIXSYNC {div}")
  print("""With this application, you can accurately find and sort songs by key and BPM,
  curated with meticulous verification, and filter them by various attributes for a
  seamless music-matching experience.""")

  surrounding_keys = []
  process_opt = [1, 2, 3, 4]

  # List of keys
  key_list = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]

  db_title = 'Western'
  db_name = 'western.csv'

  # @markdown Folder Location
  location = "/gdrive/My Drive/MixSync" # @param {type:"string"}
  location = location + "/western.csv"

  database = pd.read_csv(location)
  col_disp = ['ARTIST', 'TITLE', 'KEY', 'BPM']
  pd.set_option('max_colwidth', 30)
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)

  avail_column = database.columns.values

  print(f"SEARCH FIELDS: {avail_column}")

  song_dl = {}
  artist = "Sabrina Carpenter" # @param {type:"string"}
  cont_artist = "" # @param {type:"string"}
  title = "Espresso" # @param {type:"string"}
  key = "C" # @param ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
  bpm = 105 # @param {type:"slider", min:70, max:139, step:1}
  s_type = "Western"
  year = 2024 # @param {type:"integer"}

  if avail_column[counter] == 'KEY':
      if song_data not in key_list:
          print("Invalid key.")
          exit()

  elif avail_column[counter] == 'BPM':
      song_data = int(song_data)
      while True:
          if song_data < 70:
              song_data *= 2
          elif song_data >= 140:
              song_data /= 2
          elif 70 < song_data < 140:
              break

  elif avail_column[counter] == 'YEAR':
      song_data = int(song_data)
      while True:
          if 0 < song_data:
              break
          else:
              print("Invalid year.")
              exit()

  song_dl = {
      'ARTIST': artist,
      'CONTRIBUTING ARTIST': cont_artist ,
      'TITLE': title,
      'KEY': key,
      'BPM': bpm,
      'TYPE': s_type,
      'YEAR': year
  }

  print(song_dl)

  for item in range(len(avail_column)):
      current_col = avail_column[item]
      current_data = song_dl[current_col]
      print(f"[{item + 1}] {current_col}: {current_data}")

  new_data = pd.DataFrame([song_dl])
  data_check = input("Is this correct [Y/N]: ").lower()

  if data_check == "y":
      updated_df = pd.concat([database, new_data], ignore_index=True)
      updated_df.to_csv(location, index=False)

  else:
      data_confirm = input("Discard the current record? [Y/N]:").lower()
      if data_confirm == "y":
          print("Data discarded.")
      elif data_confirm == "n":
          updated_df = pd.concat([database, new_data], ignore_index=True)
          updated_df.to_csv(location, index=False)

=============== WELCOME TO MIXSYNC ===============
With this application, you can accurately find and sort songs by key and BPM,
  curated with meticulous verification, and filter them by various attributes for a
  seamless music-matching experience.
SEARCH FIELDS: ['ARTIST' 'CONTRIBUTING ARTIST' 'TITLE' 'KEY' 'BPM' 'TYPE' 'YEAR']
{'ARTIST': 'Sabrina Carpenter', 'CONTRIBUTING ARTIST': '', 'TITLE': 'Espresso', 'KEY': 'C', 'BPM': 105, 'TYPE': 'Western', 'YEAR': 2024}
[1] ARTIST: Sabrina Carpenter
[2] CONTRIBUTING ARTIST: 
[3] TITLE: Espresso
[4] KEY: C
[5] BPM: 105
[6] TYPE: Western
[7] YEAR: 2024
Is this correct [Y/N]: y
